# POB Lab 8 Operacje morfologiczne

In [ ]:
#from google.colab.patches import cv2_imshow
import cv2
import numpy as np
from matplotlib import pyplot as plt

#!wget https://github.com/knave88/APO/raw/main/A1.bmp -O A1.bmp
#!wget https://github.com/knave88/APO/raw/main/A1_result.bmp -O A1_result.bmp
#!wget https://github.com/knave88/APO/raw/main/testing.bmp -O testing.bmp

# Wstęp

In [ ]:
# Wczytanie obrazu pierwotnego
img = cv2.imread('A1_result.bmp', cv2.IMREAD_GRAYSCALE)
#cv2_imshow(img)
plt.figure(figsize=(12,12))
plt.imshow(img, cmap=plt.get_cmap('gray'))

## Elementy struturalne
Właściwości filtrów morfologicznych określane są przez tzw. element strukturalny, wykorzystywany jako ruchome okno. Określony jest względem wybranego piksela, tzw. punktu centralnego lub początkowego. Dla ułatwienia przyjmuje się standardowo za punkt początkowy środkowy piksel okna. Z tego też względu rozmiar elementu struktualnego powinien być liczbą nieparzystą.

In [ ]:
# element strukturalny o kształcie kwadratu 5x5
cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))

In [ ]:
# element strukturalny o kształcie prostokąta 2x3
cv2.getStructuringElement(cv2.MORPH_RECT,(2,3))

In [ ]:
# element strukturalny o kształcie elipsy 3x3 (bardzo zgrubne przybliżenie, takie same jak romb)
cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))

In [ ]:
# element strukturalny o kształcie elipsy 5x5 (już wyraźnie widać kształt eliptyczny)
cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))

In [ ]:
# element strukturalny o kształcie krzyża 5x5 
cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))

Element strukturalny o kształcie rombu nie jest zaimplementowany w OpenCV, natomiast dość łatwo można to zrobic samemu. Poniżej implementacja definująca romb o zadanym promieniu (licząc z pominięciem punktu centralnego).

In [ ]:
def diamond(r):
    return np.uint8(np.add.outer(*[np.r_[:r,r:-1:-1]]*2)>=r)

In [ ]:
diamond(2)

In [ ]:
diamond(3)

## Operacje morfologiczne

## a) erozja

Erozja jest jednym z podstawowych przekształceń morfologicznych. Erozja, czyli zwężanie, jest zastosowaniem różnicy Minkowskiego do obrazu. Jej działanie polega na obcinaniu brzegów obiektu na obrazie. Jeżeli choć jeden piksel z sąsiedztwa objętego przez kernel ma wartość równą zero, punkt centralny również otrzymuje wartość zero. W przeciwnym wypadku jego wartość nie ulega zmianie.

In [ ]:
# ustawienie elementu strukturalnego (kernel) 
kernel = np.ones((5,5),np.uint8)
# wykonanie erozji morfologicznej przy użyciu zdefiniowanego elementu strukturalnego 
img_erode = cv2.erode(img, kernel, iterations = 2, borderType = cv2.BORDER_REPLICATE)
# w tym przypadku wykonujemy dwie iteracje czyli powtarzamy wykonanie tej operacji, dla bardziej wyrażnego rezultatu
# podobnie jak wprzypadku operacji sąsiedztwa z Lab3 mozemy podać argument borderType określjący przetwarzanie pikseli na krawędziach obrazu

In [ ]:
# porównanie obrazu oryginalnego i po operacji morfologicznej
frame = cv2.hconcat((img, img_erode))
#cv2_imshow(frame)
plt.figure(figsize=(15,15))
plt.imshow(frame, cmap=plt.get_cmap('gray'))

Jak widać operacja erozji powoduje zmniejszenie obiektów. Ponieważ zastosowane dwie iteracje operacji erozji z dość dużym elementem strukturalnym to zmniejszenie obiektów jest bardzo zauważalne. Ponadto, część małych obiektów zniknęła całkowicie. W zależności od zastosowania może to być efekt pozytywny lub nie.

## b) dylacja

Dylatacja, czyli rozszerzanie, jest zastosowaniem sumy Minkowskiego do obrazu.  Jeżeli choć jeden piksel z sąsiedztwa objętego przez kernel ma wartość równą jeden, punkt centralny również otrzymuje wartość jeden. W przeciwnym wypadku przypisywane jest mu zero. 

Dylatacja służy do zamykania małych otworów oraz zatok we wnętrzu figury. Obiekty zwiększają swoją objętość i jeśli dwa lub więcej obiektów położonych jest blisko siebie, zrastają się w większe obiekty.

In [ ]:
# wykonanie erozji morfologicznej przy użyciu zdefiniowanego elementu strukturalnego 
img_dilate = cv2.dilate(img, kernel, iterations = 2, borderType = cv2.BORDER_REPLICATE)

frame = cv2.hconcat((img, img_dilate))
#cv2_imshow(frame)
plt.figure(figsize=(15,15))
plt.imshow(frame, cmap=plt.get_cmap('gray'))

Jak widać operacja dylacji powoduje powiększenie obiektów. Należy zwrócić uwagę na kształt krawędzi obiektów który jest zależny od kształtu elementu strukturalnego. Im element strukturalny ma ostre krawędze tym bardziej "kanciaste" będą obiekty. Im bardziej element strukturalny przypomina koło tym bardziej obłe będą krawędzie obiektów.

## c) otwarcie mofologiczne

Otwarcie morfologiczne jest równoważne nałożeniu operacji dylatacji na wynik erozji obrazu pierwotnego. Dzięki zastosowaniu dwóch podstawowych operacji w parze możliwe jest pozbycie się niedogodności związanej ze zmianą rozmiaru obiektów. 

Operacja otwarcia zachowuje rozmiary obiektów obrazu przy ich jednoczesnym wygładzeniu – usunięciu wszystkich „wystających” elementów. Często używana do usuwania małych obiektów (artefaktów/"śmieci") z obrazu.

In [ ]:
# wykonanie operacji otwarcia morfologicznego przy użyciu zdefiniowanego elementu strukturalnego 
img_open = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel, borderType = cv2.BORDER_REPLICATE)

frame = cv2.hconcat((img, img_open))
#cv2_imshow(frame)
plt.figure(figsize=(15,15))
plt.imshow(frame, cmap=plt.get_cmap('gray'))

## d) zamknięcie morfologiczne

Operacja zamknięcia (czasem zwana domknięciem) morfologicznego jest równoważna nałożeniu operacji erozji na wynik dylatacji obrazu pierwotnego. Domknięcie usuwa z obrazu „dziury” oraz wklęsłości mniejsze od elementu strukturalnego. Uwaga, obietky mogą się połączyć.

In [ ]:
# wykonanie operacji zamknięcia morfologicznego przy użyciu zdefiniowanego elementu strukturalnego 
img_close = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel, borderType = cv2.BORDER_REPLICATE)

frame = cv2.hconcat((img, img_close))
#cv2_imshow(frame)
plt.figure(figsize=(15,15))
plt.imshow(frame, cmap=plt.get_cmap('gray'))

# Zadanie 1

Dla wskazanego obrazu wykonać operację erozja i dylacja z podanymi elementami strukturalnymi o zadanym rozmiarze:


In [ ]:
# wczytanie obrazu do zadania 1
img1 = cv2.imread('digits1_half.bmp', cv2.IMREAD_GRAYSCALE)
#cv2_imshow(img)
plt.figure(figsize=(15,15))
plt.imshow(frame, cmap=plt.get_cmap('gray'))

In [ ]:
# widget do konstrukcji elementu strukturalnego

from ipywidgets import widgets, GridspecLayout, BoundedIntText, Layout, ToggleButton

grid_size = 7

def create_expanded_input():
    return widgets.ToggleButton(value=False, description='Click me',disabled=False, button_style='', icon='check', style={'button_width' : '100px'})

grid = GridspecLayout(grid_size, grid_size)

for i in range(grid_size):
    for j in range(grid_size):
        grid[i, j] = create_expanded_input();

print("Budowa elementu strukturalnego - klikajac odpowiednie przyciski zmieniamy wartość")
grid


In [ ]:
# zczytanie kerenela z widget'a
kernel = np.zeros((grid_size,grid_size))
for i in range(grid_size):
    for j in range(grid_size):
        kernel[i, j] = int(grid[i, j].value)

kernel = kernel.astype(np.uint8)


# utworzenie widget listy rozwijanej op morf
my_op = widgets.Dropdown(
    options=[('Erode', 1), ('Dilate', 2), ('Open', 3), ('Close', 4)],
    value=1,
    description='Morph op:',
)

# utworzenie widget przyscisk
button = widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)

# funkcja uruchamiana po przysicnieciu guzika
def my_foo(self):
  print("Kernel:")
  print(kernel)
  print("\n")

  if (my_op.value==1):
    img_out = cv2.erode(img1, kernel, iterations = 1, borderType = cv2.BORDER_REPLICATE)
    print("Operation: Erosion")
  elif (my_op.value==2):
    img_out = cv2.dilate(img1, kernel, iterations = 1, borderType = cv2.BORDER_REPLICATE)
    print("Operation: Dilation")
  elif (my_op.value==3):
    img_out = cv2.morphologyEx(img1, cv2.MORPH_OPEN, kernel, borderType = cv2.BORDER_REPLICATE)
    print("Operation: Open")
  elif (my_op.value==4):
    img_out = cv2.morphologyEx(img1, cv2.MORPH_CLOSE, kernel, borderType = cv2.BORDER_REPLICATE)
    print("Operation: Close")
  else:
    img_out = cv2.dilate(img1, kernel, iterations = 1, borderType = cv2.BORDER_REPLICATE)

  frame = cv2.hconcat((img1, img_out))
  #cv2_imshow(frame)
  plt.figure(figsize=(15,15))
  plt.imshow(frame, cmap=plt.get_cmap('gray'))
  return frame


# przypisanie funkcji do guzika
button.on_click(my_foo)
# zdefiniowanie widgetów obok siebie
grid2 = GridspecLayout(1, 2)
grid2[0,0] = my_op
grid2[0,1] = button

In [ ]:
# widget do wykonania operacji morfologicznych
grid2

In [ ]:

#img_out = cv2.erode(img, kernel, iterations = 1, borderType = cv2.BORDER_REPLICATE)
#img_out = cv2.dilate(img, kernel, iterations = 1, borderType = cv2.BORDER_REPLICATE)
#img_out = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel, borderType = cv2.BORDER_REPLICATE)
#img_out = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel, borderType = cv2.BORDER_REPLICATE)

#frame = cv2.hconcat((img, img_out))
#cv2_imshow(frame)

# Ad. Zadanie 4 Szkieletyzacja (ścienianie) obiektów 

Szkielet figury, to zbiór wszystkich punktów równoodległych od co najmniej dwóch brzegów. Szkielet odzwierciedla podstawowe własności topologiczne figury, a jego dalsza analiza może zostać wykorzystana do wielu celów.

Szkieletyzacja jest operacją wyznaczania liniowej reprezentacji (szkieletów) figur na analizowanym obrazie. Można ją wyznaczać między innymi przy użyciu operacji morfologicznych. Poniżej przedstawiona jest bardzo uproszczona implementacja.


---


ALGORYTM:
1.   Utworzenie pustego obrazu do przechowania szkieletu (*skel*).
2.   Wykonanie operacji otwarcia morfologicznego na obrazie oryginalnym, uzyskując *im_open*. 
3.   Odjęcie im_open od obrazu oryginalnego, uzyskując *im_temp*.
4.   Wykonanie operacji erozji na oryginalnym obrazie oraz poprawienie szkieletu przez obliczenie części wspólnej *im_temp* oraz aktualnego szkieletu.
5.   Powtórzenie kroków 2-4 aż obraz oryginalny będzie pusty (kompletnie zerodowany).




In [ ]:
# Krok 1
size = np.size(img)
skel = np.zeros(img.shape, np.uint8)
im_copy = img.copy()

# utoworzenie kernela
element = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))

In [ ]:
# Pętla obejmująca kroki 2-4
while True:
    #Krok 2: Otwarcie morfologiczne
    im_open = cv2.morphologyEx(im_copy, cv2.MORPH_OPEN, element)
    #Krok 3: Odjęcie powyższego wyniku od obrazu oryginalnego
    im_temp = cv2.subtract(im_copy, im_open)
    #Krok 4: 
    #erozja morfologiczna
    im_eroded = cv2.erode(im_copy, element)
    # aktualizacja szkieletu
    skel = cv2.bitwise_or(skel,im_temp)
    # aktualizacja obrazu przetwarzanego
    im_copy = im_eroded.copy()
    #plt.figure(figsize=(10,10))
    #plt.imshow(im_copy, cmap=plt.get_cmap('gray'))                             # Odkomentuj (te i poprzednia linie) jeśli chcesz zobaczyć kolejne kroki - uwaga wiele obrazów

    #Krok 5: Przetwij pętlę jeśli nie ma już obiektów w obrazie
    if cv2.countNonZero(im_copy)==0:
        break


In [ ]:
# wizualne porównanie wyników
frame = cv2.hconcat((img, skel))
#cv2_imshow(frame)
plt.figure(figsize=(15,15))
plt.imshow(frame, cmap=plt.get_cmap('gray'))